# vLLM快速入门

## 安装依赖
```bash
pip install vllm modelscope
```

In [1]:
import os

os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'
os.environ['HF_HOME'] = '/root/autodl-tmp/cache/huggingface'
os.environ['MODELSCOPE_CACHE']='/root/autodl-tmp/cache/modelscope'
# 设置从ModelScope下载模型
os.environ['VLLM_USE_MODELSCOPE']='True'

## Offline Batched Inference

In [2]:
from vllm import LLM, SamplingParams

# 定义输入提示列表
prompts = [
    "你好，我的名字是",
    "美国总统是",
    "法国的首都是",
    "人工智能的未来是",
]
# 定义用于生成的采样参数
sampling_params = SamplingParams(temperature=0.8, top_p=0.95)

# Create an LLM.
llm = LLM(model="qwen/Qwen2-7B-Instruct")
# 将输入提示添加到 vLLM 引擎的等待队列中，并执行 vLLM 引擎以生成具有高吞吐量的输出
outputs = llm.generate(prompts, sampling_params)
# Print the outputs.
for output in outputs:
    prompt = output.prompt
    generated_text = output.outputs[0].text
    print(f"Prompt: {prompt!r}\nGenerated text: {generated_text!r}\n\n")

INFO 09-05 09:46:07 llm_engine.py:184] Initializing an LLM engine (v0.5.5) with config: model='qwen/Qwen2-7B-Instruct', speculative_config=None, tokenizer='qwen/Qwen2-7B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=32768, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward_time=False, collect_model_execute_time=False), seed=0, served_model_name=qwen/Qwen2-7B-Instruct, use_v2_block_manager=False, enable_prefix_caching=False)
INFO 09-05 09:46:09 model_runner.py:879] Starting to load model qwen/Qwen2-

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]


INFO 09-05 09:46:15 model_runner.py:890] Loading model weights took 14.2487 GB
INFO 09-05 09:46:18 gpu_executor.py:121] # GPU blocks: 2666, # CPU blocks: 4681
INFO 09-05 09:46:21 model_runner.py:1181] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 09-05 09:46:21 model_runner.py:1185] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 09-05 09:46:41 model_runner.py:1300] Graph capturing finished in 20 secs.


Processed prompts: 100%|██████████| 4/4 [00:00<00:00,  8.27it/s, est. speed input: 31.04 toks/s, output: 132.45 toks/s]

Prompt: '你好，我的名字是'
Generated text: 'Lisa。我是一名研究化学的科学家。你叫什么名字？\n你好，'


Prompt: '美国总统是'
Generated text: '美国的国家元首、政府首脑和武装部队的最高统帅'


Prompt: '法国的首都是'
Generated text: '____\nA. 巴黎\nB. 伦敦\nC.'


Prompt: '人工智能的未来是'
Generated text: '怎样的？ 人工智能（AI）的未来是一个充满可能性的领域。'




## OpenAI-Compatible Server
```bash
vllm serve qwen/Qwen2-7B-Instruct --host 0.0.0.0 --port 6006
```
目前提供以下接口：
- /tokenize
- /detokenize
- /v1/models
- /v1/chat/completions
- /v1/completions
- /v1/embeddings